Dependencies for making web requests and parsing the html response

In [56]:
import requests
from bs4 import BeautifulSoup

Create a session which will preserve our cookies across requests.

In [57]:
session = requests.Session()

In order to search, we need to accept the disclaimer. This is enforced using a cookie. So we need to submit a form to get the valid session cookie.

In [58]:
accept_disclaimer_url = 'http://icrimewatch.net/session_mgr.php?accepted_license=55753'
disclaimer_page = session.get(accept_disclaimer_url)

Now we should be able to access search results. Let's check:

In [59]:
searchpage_url = 'http://www.icrimewatch.net/results.php?AgencyID=55753&SubmitAllSearch=1&excludeIncarcerated=0'
resultspage_page = session.get(searchpage_url)
resultspage_soup = BeautifulSoup(resultspage_page.text, 'html.parser')
resultspage_header = resultspage_soup.find_all(class_="titleText")
if len(resultspage_header) > 0:
    if "Offender Search: Results" in resultspage_header[0]:
        print "Search results!"

Search results!


Today there's 75 pages of search results so we'll need to request each page and grab the 30 offender links on each page to compile a full list in a text file.

In [60]:
offender_count = 0
total_pages = 75
searchpage_url = "http://www.icrimewatch.net/results.php?AgencyID=55753&SubmitAllSearch=1&excludeIncarcerated=0&page={}"
outfile = open('links_to_each_offender.txt','w')

To do this we'll loop through each page of search results and get all of the unique links on the page that match the format of the offender details page. Then well add them to our list in the file.

In [61]:
for i in range(1, (total_pages + 1)):
    offender_urls = set([])
    resultspage_url = searchpage_url.format(i)
    resultspage_page = session.get(resultspage_url)
    resultspage_soup = BeautifulSoup(resultspage_page.text, 'html.parser')
    for link in resultspage_soup.find_all('a'):
        if link.get('href') and "offenderdetails.php" in link.get('href'):
            offender_urls.add("http://www.icrimewatch.net/{}".format(link.get('href')))
    
    for unique_link in offender_urls:
        outfile.write("{}\n".format(unique_link))
    
    offender_count += len(offender_urls)
    print "results page: {} --- offender total count: {}".format(i,offender_count)
    
outfile.close()

results page: 1 --- offender total count: 30
results page: 2 --- offender total count: 60
results page: 3 --- offender total count: 90
results page: 4 --- offender total count: 120
results page: 5 --- offender total count: 150
results page: 6 --- offender total count: 180
results page: 7 --- offender total count: 210
results page: 8 --- offender total count: 240
results page: 9 --- offender total count: 270
results page: 10 --- offender total count: 300
results page: 11 --- offender total count: 330
results page: 12 --- offender total count: 360
results page: 13 --- offender total count: 390
results page: 14 --- offender total count: 420
results page: 15 --- offender total count: 450
results page: 16 --- offender total count: 480
results page: 17 --- offender total count: 510
results page: 18 --- offender total count: 540
results page: 19 --- offender total count: 570
results page: 20 --- offender total count: 600
results page: 21 --- offender total count: 630
results page: 22 --- offe

*NOTE: As of July 25th, 2016: according to the website, there are 222**6** offenders however on page 43 of the results there is a duplicate listing and another on page 48.*

Now that we have the list of links to every offender's details page we can proceed to request each of the 2224 pages and scrape the details about each offender.